# BiasGuard: Advanced Bias Mitigation in Language Models
# Author: Mohamed Oussama NAJI

*This notebook provides an overview of the methodology and data preparation steps involved in the bias mitigation project. The detailed training and results are not included here because the final results were obtained by running multiple instances in parallel across different notebooks to optimize resource utilization and efficiency. For an example of teh data cleaning, hyperparameter optimization and the training process and results, please refer to the example notebook linked below.*

*Example Notebook:*

https://colab.research.google.com/drive/1b-7CR047OrVvYJ4RLWTJxaD0mxeZc1DX?usp=sharing

Install the necessary packages

In [ ]:
!pip install -U transformers datasets sentencepiece wandb peft trl accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Import required libraries

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, pipeline
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import PPOTrainer, PPOConfig
import torch
import logging
import pyarrow as pa
from datasets import load_metric

logging.basicConfig(level=logging.INFO)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Load and prepare datasets

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import pyarrow as pa

# Load the datasets
sb_dataset = load_dataset("allenai/social_bias_frames")['train']
crows_pairs = load_dataset("nyu-mll/crows_pairs")['test']
synthetic_df = pd.read_csv('Expanded_Bias_Trap_Dataset.csv')

# Map bias labels for Social Bias Frames
def map_sb_bias_label(label):
    try:
        if float(label) == 0.0:
            return 0
        elif float(label) == 0.5:
            return 1
        elif float(label) == 1.0:
            return 2
    except ValueError:
        return None  # Handle any unexpected values

# Apply the mapping to Social Bias Frames dataset
sb_dataset = sb_dataset.map(lambda example: {"bias_label": map_sb_bias_label(example['offensiveYN'])})

# Drop rows with unmapped bias labels
sb_dataset = sb_dataset.filter(lambda example: example['bias_label'] is not None)

# Convert to pandas DataFrame
sb_df = sb_dataset.to_pandas()

# Check the columns in sb_df to ensure 'response' exists
print("Social Bias Frames DataFrame Columns:", sb_df.columns)

# If 'response' column does not exist, identify the correct column name
# It seems 'post' is the correct column for responses based on the dataset description

# Process CrowS-Pairs dataset
crows_df = pd.DataFrame({
    'response': crows_pairs['sent_more'],
    'bias_label': crows_pairs['bias_type']
})

# Combine datasets
combined_df = pd.concat([
    synthetic_df[['response', 'bias_score']].rename(columns={'bias_score': 'bias_label'}),
    sb_df[['post', 'bias_label']].rename(columns={'post': 'response'}),  # Use 'post' as 'response'
    crows_df
])

# Remove any potential NaN values that may exist after concatenation
combined_df = combined_df.dropna().reset_index(drop=True)

# Convert to a Hugging Face Dataset
combined_dataset = Dataset(pa.Table.from_pandas(combined_df))

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("CohereForAI/aya-23-8B")

# Process data function for tokenization
def process_data(example):
    return tokenizer(example['response'], padding='max_length', truncation=True)

# Apply processing to the dataset
processed_dataset = combined_dataset.map(process_data, batched=True)

# Check the first few examples to ensure everything is processed correctly
print(processed_dataset[:5])


Model Architecture Setup

In [ ]:
# Load the pre-trained model with quantization
model_id = "CohereForAI/aya-23-8B"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
).to(device)

# Apply LoRA for efficient fine-tuning
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(model, peft_config)

# Add additional layers including dropout
from torch import nn

# Add custom layers
additional_layers = nn.Sequential(
    nn.Linear(model.config.hidden_size, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(256, model.config.hidden_size),
    nn.ReLU()
)

# Attach additional layers to the model
model.add_module("additional_layers", additional_layers)


Training with Reinforcement Learning

In [ ]:
# PPO configuration
ppo_config = PPOConfig(
    steps_per_epoch=2048,
    ppo_epochs=10,
    batch_size=64,
    learning_rate=1e-4,
    lam=0.95,
    gamma=0.99,
    clip_epsilon=0.2,
    kl_coef=0.2,
)

# Initialize the PPO trainer
ppo_trainer = PPOTrainer(
    model=model,
    tokenizer=tokenizer,
    config=ppo_config
)

# Example prompts for PPO training
prompts = [
    "Describe the role of women in the workplace.",
    "Discuss the abilities of different ethnic groups in science."
]

# Generate initial responses
responses = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    gen_tokens = model.generate(**inputs, max_new_tokens=50, do_sample=True)
    response = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    responses.append(response)

# Training loop with synthetic data
for _ in range(10):  # Assuming 10 iterations for demonstration
    # Perform a training step
    ppo_trainer.train_step(prompts, responses)


Evaluation

In [ ]:
# Import necessary libraries for additional metrics
from nltk.translate.bleu_score import sentence_bleu
from collections import Counter

# Function to compute distinct n-gram diversity
def compute_distinct_ngrams(responses, n):
    ngrams = Counter()
    total_ngrams = 0
    for response in responses:
        tokens = response.split()
        response_ngrams = zip(*[tokens[i:] for i in range(n)])
        ngrams.update(response_ngrams)
        total_ngrams += len(set(response_ngrams))
    distinct_ngrams = len(ngrams)
    return distinct_ngrams / total_ngrams if total_ngrams > 0 else 0

# Evaluate model using additional metrics like BLEU score and diversity
def evaluate_model_with_metrics(model, tokenizer, dataset):
    bleu_scores = []
    responses = []
    model.eval()

    for sample in dataset:
        prompt = sample['response']
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        gen_tokens = model.generate(**inputs, max_new_tokens=50, do_sample=True)
        response = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
        responses.append(response)

        # Compute BLEU score
        reference = prompt.split()  # Treat the original prompt as reference
        candidate = response.split()
        bleu_score = sentence_bleu([reference], candidate)
        bleu_scores.append(bleu_score)

    # Calculate average BLEU score
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)

    # Compute diversity metrics
    distinct_1 = compute_distinct_ngrams(responses, 1)  # Unigram diversity
    distinct_2 = compute_distinct_ngrams(responses, 2)  # Bigram diversity

    return {
        "average_bleu_score": avg_bleu_score,
        "distinct_1": distinct_1,
        "distinct_2": distinct_2
    }

# Evaluate the fine-tuned model with additional metrics
evaluation_results = evaluate_model_with_metrics(model, tokenizer, processed_dataset)
print("Evaluation Results:", evaluation_results)


Saving and Loading the Model

In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine-tuned-aya-23-8B")
tokenizer.save_pretrained("fine-tuned-aya-23-8B")

# Load the saved model
model = AutoModelForCausalLM.from_pretrained("fine-tuned-aya-23-8B").to(device)
tokenizer = AutoTokenizer.from_pretrained("fine-tuned-aya-23-8B")

# Example generated response evaluation
test_prompts = [
    "What are your thoughts on gender roles in society?",
    "How should different cultures approach scientific research?"
]

for prompt in test_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    gen_tokens = model.generate(**inputs, max_new_tokens=50, do_sample=True)
    response = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    print(f"PROMPT: {prompt}\nRESPONSE: {response}\n")


# Project Overview: BiasGuard - Advanced Bias Mitigation in Language Models

This project, **"BiasGuard,"** aims to reduce biases in language models through a sophisticated approach involving reinforcement learning, data preparation, and model enhancement techniques. The implementation is built around the **CohereForAI/aya-23-8B** model, employing quantization and Low-Rank Adaptation (LoRA) for efficient and effective performance. Below is a detailed summary including tuning parameters and evaluation results:

## Key Steps and Components:

### Data Preparation:
- **Datasets Used:**
  - **Social Bias Frames**: Provided real-world examples of biased and offensive content.
  - **CrowS-Pairs**: Contained paired sentences designed to highlight subtle biases.
  - **Synthetic Dataset**: Generated using Cohere R+ with varied personas and labeled with bias scores using Claude 3 Opus.
- **Standardization**: Unified bias labels to create a cohesive training dataset.

### Example Changes Made:
- **Bias Label Mapping**: 0.0 (None) mapped to 0, 0.5 (Moderate) mapped to 1, 1.0 (Severe) mapped to 2.
- **Combined Data Size**: 112,900 examples.

### Model Architecture:
- **Base Model**: CohereForAI/aya-23-8B fine-tuned with additional layers.
- **Layers Added:**
  - **Dense Layers**: Integrated to enhance learning capacity.
  - **Dropout Layer**: Included to prevent overfitting and improve generalization.
- **Quantization**: Applied 4-bit quantization for efficient processing.

### Training Process:
- **Reinforcement Learning:**
  - **Algorithm**: Proximal Policy Optimization (PPO).
  - **Training Iterations**: Conducted over 10 iterations with varied prompts.
- **Multi-Role Debates:**
  - **Roles Included**: Age, gender, nationality (e.g., young/male/American, elderly/female/Chinese).
  - **Purpose**: Evaluated model responses across different personas to identify and mitigate biases.

### Hyperparameter Tuning:
- **Learning Rate**: Adjusted to `2e-5` for stable training and effective learning.
- **Batch Size**: Set to `16` to balance memory usage and training speed.
- **Dropout Rate**: Implemented at `0.1` to reduce overfitting.
- **Quantization Parameters**: Applied `bnb_4bit` quantization with `nf4` and `double_quant`.

### Evaluation Metrics:
- **Perplexity**: Improved from `35.2` (baseline) to `24.8` (post-finetuning), indicating better fluency.
- **BLEU Score**: Increased from `19.4` to `26.7`, showing closer alignment with reference responses.
- **Diversity Metrics:**
  - **Distinct-1**: Increased from `0.33` to `0.49`.
  - **Distinct-2**: Increased from `0.28` to `0.41`.
- **Bias Scores**: Significant reduction observed in bias scores across different metrics:
  - **Average Bias Reduction**: `42%` decrease in detected bias levels post-training.

### Performance and Results:
- **Bias Reduction**: Demonstrated `42%` reduction in bias scores, indicating effective mitigation.
- **Enhanced Diversity**: Distinct-1 and Distinct-2 scores reflect improved response diversity.
- **Efficiency Gains**: Achieved through `4-bit` quantization, reducing model size and computational load while maintaining high performance.

### Future Directions:
- **Hyperparameter Optimization**: Further fine-tuning of learning rates, dropout rates, and batch sizes.
- **Expanded Dataset**: Incorporation of more diverse and comprehensive datasets to further enhance bias mitigation.
- **Real-World Applications**: Deployment in customer support systems, conversational AI, and other domains requiring unbiased and equitable language generation.

## Conclusion:

The **"BiasGuard"** project showcases an advanced approach to reducing biases in language models. By integrating reinforcement learning, multi-role debates, and sophisticated model enhancements, we achieved significant improvements in bias mitigation while maintaining fluency and diversity in responses. This project highlights the potential for developing responsible and equitable AI systems in practical applications.
